In [1]:
#importing required library
import os
import cv2
import numpy as np

In [2]:
# total training image number
a = os.listdir(os.getcwd())
total_tr = len(os.listdir(os.getcwd()+"\\training"))-1
print(total_tr)

12


In [3]:
b = os.listdir(os.getcwd()+"\\testing")
total_te = len(os.listdir(os.getcwd()+"\\testing"))
print(total_te)

3


In [4]:
lookup = os.listdir(os.getcwd()+"\\training")[1:]

ans = []

for im in lookup:
    ans.append(im.split("_")[0].split("-")[0])

print(ans)

test_answer = []

for t in b:
    test_answer.append(t.split("_")[0].split("-")[0])
print(test_answer)

['parth', 'parth', 'parth', 'parth', 'parth', 'parth', 'yash', 'yash', 'yash', 'yash', 'yash', 'yash']
['parth', 'yash', 'yash']


In [ ]:
#initializing global parameters
#folder_tr = "C:\Users\ynebh\OneDrive\Pictures\Camera Roll" # name of folder containing train images 
#folder_te = "color/test"  # name of folder containing test images
#total_tr = len(next(os.walk(os.getcwd()+"/"+folder_tr))[2])
#total_te = len(next(os.walk(os.getcwd()+"/"+folder_te))[2])




In [5]:
dim = 100
threshold = 90

In [ ]:
#preparing lookup array
#lookup = []
#def fn(text):
#    ans=[]
#    for x in text:
#        if x == '_':
#            return (ans)
#        elif x== 'O':
#            return (ans)
#        elif x == 'o':
#            return (ans)
#        else:
#            ans+=x
#            
#folder="cc"
#for filename in os.listdir(os.getcwd()+"/"+folder_tr):
#    ans = fn(filename)
#    ans = ''.join(str(e) for e in ans)
#    lookup.append(ans)
#

#len(lookup)

1. Image resize
2. Image gray scale conversion
    COLOR: R, G, B
    Black and White: 0 and 255 
    Gray scale: [0, 255]
3. Image flatten
4. Create Flattened Image matrix (contain all images in vector form)

In [6]:
#L is set of all training images
#Currently L is zero matrix
import numpy as np
L = np.empty(shape=(dim*dim,total_tr), dtype='float64')
print(L.shape)
i = 0
import os,cv2

for filename in os.listdir(os.getcwd()+"\\training")[1:]:
    print(filename)
    #Read a image 
    image = cv2.imread(os.getcwd()+"\\training"+"/"+filename)
    print(image)
    resized = cv2.resize(image, (dim,dim), interpolation = cv2.INTER_AREA)
    #lookup.append(filename)
    #Convert it into grayscale image
    gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    #flattening an image
    
    gray_vector = np.array(gray_image, dtype='float64').flatten()
    L[:, i] = gray_vector[:] 
    #temp.append(gray_vector)
    i = i + 1

print("done")

(10000, 12)
parth_2018-09-30-234217.jpg
[[[128 123 152]
  [127 122 151]
  [127 122 151]
  ...
  [107 128 159]
  [104 128 158]
  [103 127 157]]

 [[128 123 152]
  [127 122 151]
  [127 122 151]
  ...
  [107 129 157]
  [106 127 158]
  [105 126 157]]

 [[127 123 152]
  [127 123 152]
  [128 124 153]
  ...
  [108 129 157]
  [106 128 156]
  [105 127 155]]

 ...

 [[105 115 149]
  [105 115 149]
  [107 115 152]
  ...
  [119 121 139]
  [119 121 139]
  [119 121 139]]

 [[104 114 148]
  [104 114 148]
  [106 114 151]
  ...
  [119 121 139]
  [119 121 139]
  [119 121 139]]

 [[103 113 147]
  [104 114 148]
  [106 114 151]
  ...
  [119 121 139]
  [119 121 139]
  [119 121 139]]]
parth_2018-09-30-234223.jpg
[[[113 120 153]
  [114 121 154]
  [117 122 153]
  ...
  [117 128 155]
  [116 127 154]
  [116 127 154]]

 [[115 120 153]
  [116 121 154]
  [117 122 153]
  ...
  [117 128 155]
  [116 127 154]
  [116 127 154]]

 [[115 120 153]
  [115 120 153]
  [117 122 153]
  ...
  [117 128 155]
  [116 127 154]
  [116 1

In [ ]:
L.shape

In [7]:
#Mean face
mean_face = np.sum(L, axis=1) /total_tr

# # mean_face = np.matrix(mean_face)
# # mean_face = mean_face.T
mean_face[:]

#subtract mean face
for i in range(total_tr):
    L[:,i] -= mean_face[:]

print("L",L.shape)
print("LT",L.T.shape)
print("mean face",mean_face.shape)

LT = L.transpose()

#calculate LTL: It is covariance matrix
C = np.matrix(LT) * np.matrix(L)  
C = C/total_tr
print("Covariance (LT)(L)",C.shape)

#calculate eigenvector and eigenvalue of covariance matrix
evalues,evectors = np.linalg.eig(C)

#getting correct ordering
sort_indices = evalues.real.argsort()[::-1]

evalues = evalues.real[sort_indices]
evectors = evectors[sort_indices]

#Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(evalues[i], evectors[:,i]) for i in range(len(evalues))]
print("done")

L (10000, 12)
LT (12, 10000)
mean face (10000,)
Covariance (LT)(L) (12, 12)
done


In [9]:
#decide value of K 
import plotly
import plotly.plotly as py
from plotly.graph_objs.layout import *
from plotly.graph_objs import Bar, Scatter, Layout, Figure
import plotly.tools as tls

tot = sum(evalues)
var_exp = [(i / tot)*100 for i in sorted(evalues, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

K = 0
#print(abs(cum_var_exp))
for i in range(len(cum_var_exp)):
    K = i
    print(K)
    if(int(cum_var_exp[i])>=threshold):
        break
print(K)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,total_tr)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,total_tr)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = [trace1, trace2]

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
plotly.offline.plot(fig)
print("done")

0
1
1
done


In [ ]:
K

In [10]:
#taking K eigenvector and putting into cols of P
P = eig_pairs[0][1]
for j in range(K-1):    
    b = eig_pairs[j+1][1]
    P = np.hstack((P,b)) 
print("done")

done


In [11]:
#projecting
evectors = L * P 

# #normalize eigenvectors
norms = np.linalg.norm(evectors, axis=0)
norms.shape
evectors = evectors / norms 
# #computing weights
W = evectors.transpose() * L  
W.shape
print("done")

done


In [13]:
#classification
i = 0
wrong = 0
for filename in os.listdir(os.getcwd()+"\\testing"):
    #ans = fn(filename)
    #ans = ''.join(str(e) for e in ans)
    #test_no.append(ans)
    #print("i ",i,"test_no ",test_no[0])
    test = cv2.imread(os.getcwd()+"\\testing"+"\\"+filename)    # read as a grayscale image
    test = cv2.resize(test, (dim,dim), interpolation = cv2.INTER_AREA)
    test= cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    img_col = np.array(test, dtype='float64').flatten()  
    #print(img_col.shape)
    img_col -= mean_face                                           # subract the mean column
    img_col = np.reshape(img_col, (dim*dim, 1))                             # from row vector to col vector

    S = evectors.transpose() * img_col                                 # projecting the normalized probe onto the
                                                                            # Eigenspace, to find out the weights

    diff = W - S                                                       # finding the min ||W_j - S||
    norms = np.linalg.norm(diff, axis=0)

    closest_face_id = np.argmin(norms)
    #print(closest_face_id)
    print("Name of testing image: ", filename)
    print("Matched training image name: ",lookup[closest_face_id])
    print("True label of testing image: ",test_answer[i])
    #print(test_no[i]+" and "+lookup[closest_face_id])
    if(test_answer[i]!=ans[closest_face_id]):
        wrong = wrong + 1
    i = i + 1

print("done")

Name of testing image:  parth_2018-09-30-234252.jpg
Matched training image name:  parth_2018-09-30-234232.jpg
True label of testing image:  parth
Name of testing image:  yash_20180930_20_39_16_Pro.jpg
Matched training image name:  yash_20180930_20_39_04_Pro (2).jpg
True label of testing image:  yash
Name of testing image:  yash_20180930_20_39_17_Pro.jpg
Matched training image name:  yash_20180930_20_39_01_Pro.jpg
True label of testing image:  yash
done


In [14]:
#Accuracy
print(wrong)
(1 - wrong/len(b))*100

0


100.0

In [ ]:
# #making N*N matrix for each from N^2 * 1 image
# import matplotlib.pyplot as plt
# for i in range(K):
#     gray_vector1 = np.reshape(Y.T[i], (1000, 1000))
#     #plt.imshow(gray_vector1,cmap='gray')
    #plt.show()